In [259]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [260]:
path = r'C:\Users\eitanb\Documents\GitHub\ML_Football_DS18\DATA'

In [261]:
# teams 


#### _import data 

In [262]:
# Define file paths again
file_paths = {
    "players": f"{path}/players.csv",
    "shots": f"{path}/shots.csv",
    "teams": f"{path}/teams.csv",
    "teamstats": f"{path}/teamstats.csv",
    "appearances": f"{path}/appearances.csv",
    "games": f"{path}/games.csv",
    "leagues": f"{path}/leagues.csv",
}

# Load datasets with appropriate encoding
datasets = {}
for name, path in file_paths.items():
    try:
        datasets[name] = pd.read_csv(path, encoding="latin1")
    except Exception as e:
        print(f"Error loading {name}: {e}")

#### create Game - team- player table (for future merging)

In [263]:
import pandas as pd
# Load datasets
appearances = datasets["appearances"]
games = datasets["games"]

# Merge appearances with games on gameID
merged_df = appearances.merge(games, on="gameID", how="left")


def assign_teams(df):
    """
    Assign teams based on positionOrder.
    - The first sequence (1-17) belongs to one team.
    - When positionOrder resets to 1, the second team starts.
    """
    team_labels = []
    current_team = 1  # Start with the first team
    
    for idx, row in df.iterrows():
        if row["positionOrder"] == 1 and len(team_labels) >= 1:
            current_team = 2  # Switch to second team when GK (1) appears again
        team_labels.append(current_team)
    
    df["teamAssignment"] = team_labels
    return df
merged_df = merged_df.groupby("gameID", group_keys=False).apply(assign_teams)
merged_df["teamID"] = merged_df.apply(
    lambda row: row["homeTeamID"] if row["teamAssignment"] == 1 else row["awayTeamID"], axis=1
)


# Step 2: Handle substitutions
merged_df["wasSubstitutedOut"] = merged_df["substituteOut"].notnull()

# Step 3: Verify positionOrder 12-17 assignment using substitution data
merged_df["isValidSubstitute"] = merged_df.apply(
    lambda row: row["wasSubstitutedOut"] if 12 <= row["positionOrder"] <= 17 else True, axis=1
)

# Step 4: Ensure players are assigned to only one team per game
merged_df["teamConsistencyCheck"] = merged_df.groupby(["playerID", "gameID"])["teamID"].transform("nunique") == 1

# Step 5: Verify goals consistency
def verify_goals(merged_df, games):
    team_goals = merged_df.groupby(["gameID", "teamAssignment"])["goals"].sum().reset_index(name="teamGoals")
    own_goals = merged_df.groupby(["gameID", "teamAssignment"])["ownGoals"].sum().reset_index(name="ownGoals")
    
    team_goals = team_goals.merge(own_goals, on=["gameID", "teamAssignment"], how="left").fillna(0)
    team_goals["adjustedGoals"] = team_goals["teamGoals"] + team_goals.groupby("gameID")["ownGoals"].shift(-1, fill_value=0)
    
    team_goals = team_goals.merge(games[["gameID", "homeTeamID", "awayTeamID", "homeGoals", "awayGoals"]], on="gameID", how="left")
    
    team_goals["teamID_after_Verify"] = team_goals.apply(
        lambda row: row["homeTeamID"] if row["adjustedGoals"] == row["homeGoals"] else row["awayTeamID"], axis=1
    )
    # Merge verification results
    merged_df = merged_df.merge(team_goals[["gameID", "teamAssignment", "teamID_after_Verify"]], on=["gameID", "teamAssignment"], how="left")
    
    # Extract only rows where teamID changed
    discrepancies = merged_df[(merged_df["teamID"] != merged_df["teamID_after_Verify"])&(merged_df['homeGoals']!=merged_df['awayGoals'])][["gameID", "playerID", "teamID", "teamID_after_Verify"]]

    return discrepancies

goal_verification = verify_goals(merged_df, games)

# Step 6: Ensure a player is not assigned to more than 3 different teams in a season
player_team_counts = merged_df.groupby(["playerID", "season"])["teamID"].nunique()
valid_players = player_team_counts.index
merged_df = merged_df[merged_df.set_index(["playerID", "season"]).index.isin(valid_players)]

# Step 7: Generate a file with gameID, teamID, playerID for verification
player_game_team_mapping = merged_df[["gameID", "teamID", "playerID"]].drop_duplicates()


C:\Users\eitanb\AppData\Local\Temp\ipykernel_26300\1745827741.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby("gameID", group_keys=False).apply(assign_teams)


In [264]:
player_game_team_mapping

,gameID,teamID,playerID
0,81,89,560
1,81,89,557
2,81,89,548
3,81,89,628
4,81,89,1006
...,...,...,...
356508,16135,179,3509
356509,16135,179,4882
356510,16135,179,5786
356511,16135,179,8997


#### players - apperance and shots

In [265]:
# Prepare data for indices calculation
shots_df=datasets["shots"].copy()
appearances_df=datasets["appearances"].copy()
# Merge appearances with shots data based on gameID and playerID (shooterID)
player_shots = shots_df.rename(columns={"shooterID": "playerID"})
merged_df_teams = appearances_df.merge(player_game_team_mapping, on=["gameID", "playerID"], how="left")
merged_df_player = player_shots.merge(player_game_team_mapping, on=["gameID", "playerID"], how="left")


# Define indices calculations
player_performance = merged_df_player.groupby(["gameID","teamID"]).agg(

    total_xGoals=("xGoal", "sum"),
    total_blocked_shots=("shotResult", lambda x: (x == "BlockedShot").sum()),
    total_saved_shots=("shotResult", lambda x: (x == "SavedShot").sum()),
)
# Define indices calculations
team_performance = merged_df_teams.groupby(["gameID","teamID"]).agg(

    teamgoals=("goals", "sum"),
    ownGoals=('ownGoals',"sum"),
    total_shots=("shots", "sum"),
    total_xGoal=("xGoals", "sum"),
    total_assists=("assists", "sum"),
    total_xAssists=("xAssists", "sum"),
    total_key_passes=("keyPasses", "sum"),
    total_xGoalsChain=("xGoalsChain", "sum"),
    total_xGoalsBuildup=("xGoalsBuildup", "sum"),
    total_yellow_cards=("yellowCard", "sum"),
    total_red_cards=("redCard", "sum"),
)

merged_df_teams = team_performance.merge(player_performance, on=["gameID", "teamID"], how="left").reset_index()
# Calculate per 90-minute indices
#player_performance["minutes_per_game"] = player_performance["total_minutes"] / player_performance["games_played"]
#player_performance["goal_per_90"] = (player_performance["total_goals"] / player_performance["total_minutes"]) * 90
#player_performance["assist_per_90"] = (player_performance["total_assists"] / player_performance["total_minutes"]) * 90
#player_performance["key_passes_per_90"] = (player_performance["total_key_passes"] / player_performance["total_minutes"]) * 90
#player_performance["xGoals_per_90"] = (player_performance["total_xGoals"] / player_performance["total_minutes"]) * 90
#player_performance["xAssists_per_90"] = (player_performance["total_xAssists"] / player_performance["total_minutes"]) * 90

# Shooting efficiency
#merged_df_teams["shot_accuracy"] = merged_df_teams["total_goals"] / merged_df_teams["total_shots"]
#merged_df_teams["xGoals_efficiency"] = merged_df_teams["total_goals"] / merged_df_teams["total_xGoals"]

# Defensive & GK indices
#merged_df_teams["blocked_shots_per_90"] = (merged_df_teams["total_blocked_shots"] / 90) * 90
#player_performance["saved_shots_per_90"] = (player_performance["total_saved_shots"] / player_performance["total_minutes"]) * 90
#player_performance["yellow_cards_per_game"] = player_performance["total_yellow_cards"] / player_performance["games_played"]
#player_performance["red_cards_per_game"] = player_performance["total_red_cards"] / player_performance["games_played"]

#merged_df_teams = team_performance.merge(player_performance, on=["gameID", "teamID"], how="left").reset_index()


In [266]:
teamstats=datasets["teamstats"].merge(merged_df_teams,on=['gameID', 'teamID'],how='left')

#### merge games and teams statistic for teame base

In [267]:
datasets["games"]=datasets["games"][["gameID","leagueID","season","date","homeTeamID","awayTeamID","homeGoals","awayGoals","homeGoalsHalfTime","awayGoalsHalfTime"]].copy()
teamstats
# Ensure location column exists in teamstats and filter accordingly
teamstats_home = teamstats[teamstats["location"] == "h"].copy()
teamstats_away = teamstats[teamstats["location"] == "a"].copy()

# Rename columns to distinguish home and away stats
teamstats_home = teamstats_home.drop(columns=["location"])
teamstats_away = teamstats_away.drop(columns=["location"])

teamstats_home.columns = [col + "_h" if col != "gameID" else col for col in teamstats_home.columns]
teamstats_away.columns = [col + "_a" if col != "gameID" else col for col in teamstats_away.columns]

# Merge correctly based on gameID and teamID for home and away teams
merged_df_teames= datasets["games"].merge(
    teamstats_home, left_on=["gameID", "homeTeamID"], right_on=["gameID", "teamID_h"], how="left"
).drop(columns=["teamID_h"])

merged_df_teames = merged_df_teames.merge(
    teamstats_away, left_on=["gameID", "awayTeamID"], right_on=["gameID", "teamID_a"], how="left"
).drop(columns=["teamID_a"])

In [276]:
teame_results=merged_df_teames[['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID',
       'homeGoals', 'awayGoals', 'homeGoalsHalfTime', 'awayGoalsHalfTime',
         'xGoals_h', 'shots_h',
       'shotsOnTarget_h', 'deep_h', 'ppda_h', 'fouls_h', 'corners_h',
       'yellowCards_h', 'redCards_h', 'teamgoals_h', 'ownGoals_h',
         'total_assists_h', 'total_xAssists_h',
       'total_key_passes_h', 'total_xGoalsChain_h', 'total_xGoalsBuildup_h',
       'total_blocked_shots_h', 'total_saved_shots_h', 'xGoals_a', 'shots_a', 'shotsOnTarget_a', 'deep_a', 'ppda_a',
       'fouls_a', 'corners_a', 'yellowCards_a', 'redCards_a',
       'teamgoals_a', 'ownGoals_a',
       'total_assists_a', 'total_xAssists_a', 'total_key_passes_a',
       'total_xGoalsChain_a', 'total_xGoalsBuildup_a',  'total_blocked_shots_a',
       'total_saved_shots_a', 'result_h']].copy()

In [277]:
teame_results

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeGoalsHalfTime,awayGoalsHalfTime,xGoals_h,shots_h,shotsOnTarget_h,deep_h,ppda_h,fouls_h,corners_h,yellowCards_h,redCards_h,teamgoals_h,ownGoals_h,total_assists_h,total_xAssists_h,total_key_passes_h,total_xGoalsChain_h,total_xGoalsBuildup_h,total_blocked_shots_h,total_saved_shots_h,xGoals_a,shots_a,shotsOnTarget_a,deep_a,ppda_a,fouls_a,corners_a,yellowCards_a,redCards_a,teamgoals_a,ownGoals_a,total_assists_a,total_xAssists_a,total_key_passes_a,total_xGoalsChain_a,total_xGoalsBuildup_a,total_blocked_shots_a,total_saved_shots_a,result_h
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,1,0,0.627539,9,1,4,13.8261,12,1,2.0,0,0,0,0,0.284979,5,1.396328,0.994160,4.0,1.0,0.674600,9,4,10,8.2188,12,2,3.0,0,0,1,0,0.586365,7,1.745371,0.811549,3.0,4.0,W
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0,0,0.876106,11,2,11,6.9000,13,6,3.0,0,0,0,0,0.419975,9,2.159510,1.170894,2.0,2.0,0.782253,7,3,2,11.8462,13,3,4.0,0,1,0,1,0.560695,4,1.238205,0.736815,2.0,2.0,L
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0,1,0.604226,10,5,5,6.6500,7,8,1.0,0,2,0,2,0.549139,8,1.025550,0.493522,2.0,3.0,0.557892,11,5,4,17.1579,13,2,2.0,0,2,0,1,0.418385,8,1.959323,1.030588,3.0,3.0,D
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,3,0,2.568030,19,8,5,10.8800,13,6,2.0,0,4,0,2,1.727543,18,6.815649,3.741916,4.0,4.0,1.459460,11,5,6,9.5556,17,3,4.0,0,2,0,2,1.288886,9,7.622863,5.617276,2.0,3.0,W
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0,1,1.130760,17,6,5,5.7368,14,1,1.0,0,1,0,1,0.416638,12,1.966623,0.699249,3.0,4.0,2.109750,11,7,10,10.6250,20,4,0.0,0,3,0,3,2.050685,10,10.799517,8.554974,2.0,4.0,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12675,16131,5,2020,2021-05-23 19:00:00,168,166,1,2,1,1,1.411190,15,5,17,12.3684,8,9,2.0,0,1,0,1,0.971853,11,3.853730,1.999150,6.0,4.0,1.707510,8,5,3,8.3529,11,5,2.0,0,2,0,1,0.307960,4,1.223212,0.715843,1.0,3.0,L
12676,16132,5,2020,2021-05-23 19:00:00,177,176,1,2,1,1,1.198190,10,3,3,16.2632,11,5,1.0,0,1,0,1,0.855524,8,1.962812,1.028432,3.0,2.0,1.238050,12,5,4,27.0000,6,2,1.0,0,2,0,1,0.775388,7,2.610665,1.758012,4.0,3.0,L
12677,16133,5,2020,2021-05-23 19:00:00,163,235,2,0,1,0,1.332690,12,6,10,8.2857,11,4,1.0,0,2,0,1,1.151649,8,7.684589,5.704923,2.0,4.0,0.357583,9,2,0,39.7273,10,3,0.0,0,0,0,0,0.216965,6,0.884652,0.544502,0.0,2.0,W
12678,16134,5,2020,2021-05-23 19:00:00,175,181,0,1,0,1,1.460500,19,5,6,7.5600,13,9,1.0,0,0,0,0,1.265829,13,4.790546,3.092978,5.0,5.0,1.380290,10,2,3,14.7200,10,3,0.0,0,1,0,1,0.565077,6,1.256511,0.764512,1.0,1.0,L


In [278]:
def game_date(df):
    # Convert 'saledate' to datetime and extract year, month, and day
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['game_year'] = df['date'].dt.year
    df['game_month'] = df['date'].dt.month
    df['game_day'] = df['date'].dt.day
    return df



In [279]:
teame_results=game_date(teame_results)

In [284]:
path = r'C:\Users\eitanb\Documents\GitHub\ML_Football_DS18\DATA'
teame_results.to_csv(path+"/"+"team_results.csv")